# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
path = "../output_data/clean_city_data.csv"
new_city_df = pd.read_csv(path)
city_df_cleaned = new_city_df.dropna()
city_df_cleaned

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Yarmouth,41.7057,-70.2286,71.19,79,1,13.00,US,1632786575
1,Ilulissat,69.2167,-51.1000,26.62,31,0,12.66,GL,1632786329
2,Faya,18.3851,42.4509,62.71,23,70,5.14,SA,1632786579
3,Warri,5.5167,5.7500,74.61,92,60,3.78,NG,1632786582
4,Nikolskoye,59.7035,30.7861,45.01,95,0,3.06,RU,1632786460
...,...,...,...,...,...,...,...,...,...
552,Kloulklubed,7.0419,134.2556,86.50,73,92,8.14,PW,1632787842
553,Krasnaya Gora,52.9995,31.6023,41.11,94,16,6.24,RU,1632787844
554,Yeppoon,-23.1333,150.7333,73.18,65,90,15.90,AU,1632787846
555,Aklavik,68.2191,-135.0107,31.21,81,7,6.82,CA,1632787849


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)

locations = city_df_cleaned[["Lat", "Lng"]]

In [10]:
humidity = city_df_cleaned["Humidity"].astype(float)

In [11]:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
cloud_df = city_df_cleaned[city_df_cleaned["Cloudiness"] == 0]
temp_df = cloud_df[(cloud_df["Max Temp"] > 70) & (cloud_df["Max Temp"] < 80)]

wind_df = temp_df[temp_df["Wind Speed"] < 10]
wind_df = wind_df.dropna()
wind_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
145,Alexandria,31.2156,29.9553,75.65,64,0,8.41,EG,1632786850
211,Nouadhibou,20.9310,-17.0347,76.98,65,0,1.14,MR,1632787052
260,Assiut,27.1810,31.1837,74.34,41,0,9.06,EG,1632787164
316,Marshall,34.3834,-86.3333,73.31,54,0,0.27,US,1632787300
369,Sabha,27.0377,14.4283,74.25,24,0,8.63,LY,1632787421
406,Tiznit Province,29.5833,-9.5000,76.37,25,0,1.99,MA,1632787508
516,Mehrān,33.1222,46.1646,79.07,13,0,8.32,IR,1632787757
539,Kumluca,36.3703,30.2869,76.30,40,0,4.09,TR,1632787812


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# Store into variable named hotel_df:
hotel_df = wind_df

# Add a "Hotel Name" column to the DataFrame: (setting new columns to hold values)
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters:

# Set params:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
    except: 
        print('Issues with finding a hotel at', row['City'])
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
hotel_df

Issues with finding a hotel at Tiznit Province
Issues with finding a hotel at Kumluca


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
145,Alexandria,31.2156,29.9553,75.65,64,0,8.41,EG,1632786850,Hilton Alexandria Corniche
211,Nouadhibou,20.9310,-17.0347,76.98,65,0,1.14,MR,1632787052,HOtel Tasiast
260,Assiut,27.1810,31.1837,74.34,41,0,9.06,EG,1632787164,"El Watania Palace Hotel, Assiut"
316,Marshall,34.3834,-86.3333,73.31,54,0,0.27,US,1632787300,Lake Guntersville Bed and Breakfast
369,Sabha,27.0377,14.4283,74.25,24,0,8.63,LY,1632787421,فندق المواجيد
406,Tiznit Province,29.5833,-9.5000,76.37,25,0,1.99,MA,1632787508,NA
516,Mehrān,33.1222,46.1646,79.07,13,0,8.32,IR,1632787757,مهمان سرا آبفا
539,Kumluca,36.3703,30.2869,76.30,40,0,4.09,TR,1632787812,NA


In [14]:
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
145,Alexandria,31.2156,29.9553,75.65,64,0,8.41,EG,1632786850,Hilton Alexandria Corniche
211,Nouadhibou,20.9310,-17.0347,76.98,65,0,1.14,MR,1632787052,HOtel Tasiast
260,Assiut,27.1810,31.1837,74.34,41,0,9.06,EG,1632787164,"El Watania Palace Hotel, Assiut"
316,Marshall,34.3834,-86.3333,73.31,54,0,0.27,US,1632787300,Lake Guntersville Bed and Breakfast
369,Sabha,27.0377,14.4283,74.25,24,0,8.63,LY,1632787421,فندق المواجيد
516,Mehrān,33.1222,46.1646,79.07,13,0,8.32,IR,1632787757,مهمان سرا آبفا


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig



Figure(layout=FigureLayout(height='420px'))